In [1]:
from datasets import load_dataset

# Tải dataset
dataset = load_dataset("OpenHust/vietnamese-summarization")

In [2]:
from collections import Counter

counter = Counter(dataset['train']['Document'])
duplicates = [doc for doc, count in counter.items() if count > 1]

print("Number of duplicated documents:", len(duplicates))

Number of duplicated documents: 14655


In [3]:
seen = set()
def dedupe(example):
    doc = example["Document"]
    if doc in seen:
        return False
    seen.add(doc)
    return True

dataset = dataset.filter(dedupe)

In [4]:
data = dataset['train'].to_pandas()

In [5]:
data.head()

,Unnamed: 0,Document,Summary,Dataset
0,0.0,Đây là một trong những nội dung tại văn bản vừ...,"Các quận , huyện , thị xã tuyên truyền bằng nh...",vietnews
1,1.0,"Chiều 12/3 , ông Vũ Hùng Triều , Trưởng phòng ...",Cháu bé được phát hiện trong tình trạng chưa c...,vietnews
2,2.0,"Thoạt đầu nhìn vào bức ảnh , nếu không có dấu ...",Báo tuyết ( Panthera uncia ) được mệnh danh là...,vietnews
3,3.0,"Khoảng 9h30 sáng 5/6 , anh Đào Nhật Tuấn ( 30 ...","Thấy con chim bị mắc bẫy rơi xuống hồ nước , a...",vietnews
4,4.0,Liên minh Tự do Dân sự Mỹ ( ACLU ) đã nhận đượ...,"Loại "" huyết thanh nói thật "" mà CIA sử dụng đ...",vietnews


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31795 entries, 0 to 31794
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  30978 non-null  float64
 1   Document    31795 non-null  object 
 2   Summary     31795 non-null  object 
 3   Dataset     31795 non-null  object 
dtypes: float64(1), object(3)
memory usage: 993.7+ KB


In [7]:
data.Dataset.value_counts().apply(lambda x: f'{x} ~ {x / data.shape[0] * 100:.2f}%')

Dataset
wikiling     17935 ~ 56.41%
vietnews     12055 ~ 37.91%
wikimulti      1664 ~ 5.23%
ViMs            141 ~ 0.44%
Name: count, dtype: object

In [8]:
data.Document.apply(len).describe()

count    31795.000000
mean      2147.108539
std        813.612712
min          1.000000
25%       1778.000000
50%       1998.000000
75%       2391.000000
max       7902.000000
Name: Document, dtype: float64

In [9]:
data.Summary.apply(len).describe()

count    31795.000000
mean       217.061330
std        217.647228
min         15.000000
25%        131.000000
50%        172.000000
75%        230.000000
max       4034.000000
Name: Summary, dtype: float64

In [10]:
filtered = data[data["Document"].apply(lambda x: len(x) == 1)]
filtered

,Unnamed: 0,Document,Summary,Dataset
26831,10530.0,.,Ong nghệ là tên gọi dùng để chỉ các loài ong t...,wikimulti


In [11]:
data = data[data["Document"].apply(lambda x: len(x) > 1)]
data = data.reset_index(drop=True)

In [12]:
data.shape

(31794, 4)

In [4]:
from vncorenlp import VnCoreNLP

vncorenlp = VnCoreNLP("../VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g')

In [14]:
def token_len(text):
    tokens = vncorenlp.tokenize(text)
    all_tokens = [token for sentence in tokens for token in sentence]
    return len(all_tokens)

In [16]:
from tqdm import tqdm

tqdm.pandas(desc="Tokenizing")
data['document_token_len'] = data['Document'].progress_apply(token_len)
data['summary_token_len'] = data['Summary'].progress_apply(token_len)

Tokenizing: 100%|██████████| 74562/74562 [20:12<00:00, 61.47it/s] 


In [22]:
data['document_token_len'].describe()

count    74562.000000
mean       403.057885
std        171.018561
min          1.000000
25%        279.000000
50%        372.000000
75%        537.000000
max       1569.000000
Name: document_token_len, dtype: float64

In [23]:
data['summary_token_len'].describe()

count    74562.000000
mean        37.047518
std         35.126790
min          4.000000
25%         24.000000
50%         30.000000
75%         39.000000
max        787.000000
Name: summary_token_len, dtype: float64

In [13]:
from datasets import DatasetDict

train_dataset = dataset['train']

train_filtered = train_dataset.filter(lambda x: len(x["Document"]) > 1)

split_1 = train_filtered.train_test_split(test_size=0.2, seed=42)
train_final = split_1["train"]
temp_test = split_1["test"]

split_2 = temp_test.train_test_split(test_size=0.5, seed=42)
validation_final = split_2["train"]
test_final = split_2["test"]

final_dataset = DatasetDict({
    "train": train_final,
    "validation": validation_final,
    "test": test_final
})

final_dataset.save_to_disk("vietnamese_summarization_split")

Filter:   0%|          | 0/31795 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25435 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3179 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3180 [00:00<?, ? examples/s]

In [1]:
from datasets import load_from_disk

splitted_dataset = load_from_disk("vietnamese_summarization_split")

In [2]:
splitted_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Document', 'Summary', 'Dataset'],
        num_rows: 25435
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'Document', 'Summary', 'Dataset'],
        num_rows: 3179
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Document', 'Summary', 'Dataset'],
        num_rows: 3180
    })
})